In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('top_rated_wines.csv')
df.head()

,name,region,variety,rating,notes
0,3 Rings Reserve Shiraz 2004,"Barossa Valley, Barossa, South Australia, Aust...",Red Wine,96.0,Vintage Comments : Classic Barossa vintage con...
1,Abreu Vineyards Cappella 2007,"Napa Valley, California",Red Wine,96.0,Cappella is a proprietary blend of two clones ...
2,Abreu Vineyards Cappella 2010,"Napa Valley, California",Red Wine,98.0,Cappella is one of the oldest vineyard sites i...
3,Abreu Vineyards Howell Mountain 2008,"Howell Mountain, Napa Valley, California",Red Wine,96.0,When David purchased this Howell Mountain prop...
4,Abreu Vineyards Howell Mountain 2009,"Howell Mountain, Napa Valley, California",Red Wine,98.0,"As a set of wines, it is hard to surpass the f..."


In [9]:
df = df[df['variety'].notna()]   #removes any NaN (not a number) values

In [29]:
data = df.sample(700).to_dict('records')
len(data)

700

In [14]:
!pip install qdrant_client
!pip install sentence_transformers

In [15]:
from qdrant_client import models, QdrantClient
from sentence_transformers import SentenceTransformer

In [30]:
#Model to create embeddings
encoder = SentenceTransformer('all-MiniLM-L6-v2')

In [31]:
#Create vector database client
qdrant = QdrantClient(":memory:")  #in-memory qdrant instance

In [33]:
#Create collection to store wines
qdrant.recreate_collection(
    collection_name="top_wines",
    vectors_config=models.VectorParams(
        size=encoder.get_sentence_embedding_dimension(),
        distance=models.Distance.COSINE
    )
)

/tmp/ipython-input-1057771960.py:2: DeprecationWarning: `recreate_collection` method is deprecated and will be removed in the future. Use `collection_exists` to check collection existence and `create_collection` instead.
  qdrant.recreate_collection(


True

In [34]:
# Vectorize
qdrant.upload_points(
    collection_name="top_wines",
    points=[
        models.PointStruct(
            id=idx,
            vector=encoder.encode(doc["notes"]).tolist(),
            payload=doc
        )
        for idx, doc in enumerate(data)
    ]
)

In [35]:
user_prompt = "Suggest me an amazing Malbech wine from Argentina"

In [43]:
# Search

hits = qdrant.search(
    collection_name="top_wines",
    query_vector=encoder.encode("99 points Cabernet Sauvignon from Napa Valley").tolist(),
    limit=3
)

for hit in hits:
  print(hit.payload, "score:", hit.score)

{'name': 'Buccella Cabernet Sauvignon 2012', 'region': 'Napa Valley, California', 'variety': 'Red Wine', 'rating': 96.0, 'notes': 'The 2012 Buccella Cabernet Sauvignon is a stunning balance between power and finesse. The wine is a lush, layered example of the best Napa Valley Cabernet Sauvignon can offer. Aromas of blackberry, cassis, and blueberry mingle with notes of molasses, bay, pastry crust, and graphite. The palate showcases a velvet texture, with supple tannins wrapped around flavors of black current, dark chocolate flavors, and a rich finish of incredible length.'} score: 0.7288671020324257
{'name': 'Hundred Acre Few and Far Between Cabernet Sauvignon 2009', 'region': 'Howell Mountain, Napa Valley, California', 'variety': 'Red Wine', 'rating': 96.0, 'notes': "Hundred Acre Few and Far Between Cabernet Sauvignon is made exclusively from one hundred percent Cabernet Sauvignon grown on Hundred Acre's estate vineyard in Napa Valley."} score: 0.6957176110526131
{'name': 'Gemstone Vi

/tmp/ipython-input-2557763746.py:3: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  hits = qdrant.search(


In [44]:
# define a variable to hold the search results
search_results = [hit.payload, "score:", hit.score]

In [40]:
!pip install openai

In [45]:
from openai import OpenAI

client = OpenAI(
    api_key="API-key"   # paste your OpenAI key here
)

In [47]:
completion = client.chat.completions.create(
    model = "gpt-4o-mini",
    messages = [
        {"role": "system", "content": "You are chatbot, a wine specialist. Your top priority is to help guide users into selecting amazing wine and guide them with their requests."},
        {"role": "user", "content": "Suggest me an amazing Malbec wine from Argentina"},
        {"role": "assistant", "content": str(search_results)}
    ]
)

print(completion.choices[0].message)

ChatCompletionMessage(content='One amazing Malbec wine from Argentina that I highly recommend is the **Catena Zapata Malbec Argentino**. \n\n### Catena Zapata Malbec Argentino\n- **Region:** Mendoza, Argentina\n- **Tasting Notes:** This wine showcases intense aromas of dark fruit such as blackberry, plum, and hints of floral notes, complemented by spice and earthiness. The palate is rich with plush tannins, balanced acidity, and a long, satisfying finish.\n- **Food Pairing:** It pairs beautifully with grilled meats, particularly steak, as well as hearty pasta dishes and aged cheeses.\n\nAnother excellent option is the **Norton Altura Malbec**:\n- **Region:** Mendoza, Argentina\n- **Tasting Notes:** This wine offers a complexity of red and black fruits, alongside velvety tannins and notes of chocolate and smoke. The vibrant acidity makes it incredibly versatile with food.\n- **Food Pairing:** Ideal for barbecue, lamb, or even savory mushroom dishes.\n\nBoth of these selections represent